# **Notebook for creating clustermaps from lineageTree data**

Importing important packages

In [ ]:
from LineageTree import lineageTree, lineageTreeManager
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import squareform
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import numpy as np

Load the manager

In [2]:


lT1 = lineageTree.load("path_to_lt/lt.lT")
lT2 = lineageTree.load("path_to_lt2/lt2.lT")
lTm = lineageTreeManager.LineageTreeManager()
lTm.add(lT1)
lTm.add(lT2)

### Or load a ready manager###

lTm = lineageTreeManager.LineageTreeManager.load("path_to/Manager.ltM")


In [3]:
lTm.lineagetrees.keys() 

{'just_orient_Points': <LineageTree.lineageTree.lineageTree at 0x7760a61cc7d0>,
 'demo2': <LineageTree.lineageTree.lineageTree at 0x77607d228c10>,
 'demo1': <LineageTree.lineageTree.lineageTree at 0x7760856ee050>}

Labels can and should be changed

In [20]:
lTm.lineagetrees["sample_lT"].labels
lTm.lineagetrees["sample_lT"].labels[id] = "name of labels"

### Select only labeled roots (Next 2 cells can be replaced by just making a list of ids)

In [5]:
roots = {}
for lt in lTm.lineagetrees:
    for root in lTm.lineagetrees[lt].labels:
        if lTm.lineagetrees[lt].labels[root]!="Unlabeled" and lTm.lineagetrees[lt].time[root]==0:
            roots.setdefault(lt, []).append(root)
roots

{'just_orient_Points': [0, 3, 12469, 233, 311, 372, 10696, 6729],
 'demo2': [176, 29345, 38764],
 'demo1': [473, 29653, 52343]}

In [14]:
time = 0 # Select a specific time point
comparisons = {}
names = {}
all_roots = [
    (
        lt,
        selected_root,
        int(node),
    )
    for lt in roots
    for node in roots[lt]
    for selected_root in lTm.lineagetrees[
        lt
    ].get_cells_at_t_from_root(int(node), time)
]
names = dict(enumerate(all_roots))
combs = combinations(names, 2)


# Otherwise create a list that contains tuples: 
[("name_of_lT", node, root),\
("name_of_lT", node, root),\
("name_of_lT", node, root),\
("name_of_lT", node, root),\
....]

### Comparison calculation

In [16]:
comparison = {}
for i, (n1,n2) in enumerate(combs):
    comparison[n1,n2] = lTm.cross_lineage_edit_distance(
                    names[n1][1],
                    names[n2][0],
                    500,
                    names[n2][1],
                    names[n2][0],
                    200,
                    style = "mini" # Best style option for speed and accuracy is "fragmented"
                )

### Calculation of the hierarchy clustermap

In [17]:
combs = combinations(names,2)
labels = [node[0][:-6]+"_"+lTm.lineagetrees[node[0]].labels[lTm.lineagetrees[node[0]].get_labelled_ancestor(node[1])] for node in names.values()]
hierarchy = np.zeros((len(names),len(names)))
keys = comparison.keys()
values = comparison.values()
for (keys,values) in  combs:
    hierarchy[keys, values] = comparison[(keys, values)]
    hierarchy[values, keys] = comparison[(keys, values)]
condensed_dist_matrix = squareform(hierarchy)
linkage_data = linkage(condensed_dist_matrix, method="ward")


In [ ]:


sns.set(rc={'figure.figsize':(30,30)}, font_scale=0.1)

sns.set_context("poster")

g = sns.clustermap(hierarchy,
            xticklabels=[label for label in labels],
            yticklabels=[label for label in labels],
            cmap="viridis",
            row_linkage=linkage_data,
            col_linkage=linkage_data,
            annot = True,
            annot_kws={"size":10}
            )

plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), color='white')
plt.setp(g.ax_heatmap.xaxis.get_majorticklabels(), color='white')

g.ax_heatmap.set_facecolor('black')
g.ax_row_dendrogram.set_facecolor('black')
g.ax_col_dendrogram.set_facecolor('black')

g.ax_heatmap.tick_params(axis='both', colors='white')

for tree in g.ax_row_dendrogram.collections:
    tree.set_color("white")
for tree in g.ax_col_dendrogram.collections:
    tree.set_color("white")

g.fig.patch.set_facecolor('black')

colorbar = g.cax
colorbar.set_facecolor('black')
colorbar.tick_params(colors='white')
colorbar.yaxis.set_tick_params(color='white')

colorbar.yaxis.label.set_color('white')


g.fig.patch.set_alpha(0.0)

g.ax_heatmap.patch.set_alpha(0.0)
g.ax_row_dendrogram.patch.set_alpha(0.0)
g.ax_col_dendrogram.patch.set_alpha(0.0)
colorbar.patch.set_alpha(0.0)
plt.show()
